In [30]:
LOCAL_IP = 'http://localhost:8000'
HEROKU_IP = 'http://www.sharelockers.com'
secret = '1' #'2' # '1' for heroku, '2' for local?

ip = HEROKU_IP


In [31]:
import requests
from pprint import pprint as pprint
from datetime import datetime
import time

In [32]:
print('### GET google.com ###')
resp = requests.get('http://www.google.com')
pprint(resp.headers)
pprint(resp.text[:255])

### GET google.com ###
{'x-frame-options': 'SAMEORIGIN', 'date': 'Sat, 11 Jul 2015 19:03:16 GMT', 'x-xss-protection': '1; mode=block', 'p3p': 'CP="This is not a P3P policy! See http://www.google.com/support/accounts/bin/answer.py?hl=en&answer=151657 for more info."', 'server': 'gws', 'content-type': 'text/html; charset=ISO-8859-1', 'expires': '-1', 'cache-control': 'private, max-age=0', 'vary': 'Accept-Encoding', 'transfer-encoding': 'chunked', 'set-cookie': 'PREF=ID=1111111111111111:FF=0:TM=1436641396:LM=1436641396:V=1:S=gzfKZblWt93IQcMc; expires=Mon, 10-Jul-2017 19:03:16 GMT; path=/; domain=.google.com, NID=69=eAVoAoG_tDep8zxRGoZzXGLXaoW1rRGuUbkRbpp2a3gazgm8rGgJlN9k09h1M89Ogxtr3aelyonUfYeqqfsUxNTf26bNl9KJ9G0tyyUB2UIqIEhPCnRLPNtfmcZDkmU5; expires=Sun, 10-Jan-2016 19:03:16 GMT; path=/; domain=.google.com; HttpOnly', 'accept-ranges': 'none', 'alternate-protocol': '80:quic,p=0'}
('<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" '
 'lang="en"><head><meta content="Se

In [33]:
print('### POST activity ###')
post_data = {'title' : str(datetime.now())}
resp = requests.post('https://vast-headland-9093.herokuapp.com/api/activities/', data=post_data)
pprint(resp.json())

### POST activity ###
{'id': 20,
 'stat_set': [],
 'title': '2015-07-11 15:03:17.071882',
 'url': 'https://vast-headland-9093.herokuapp.com/api/activities/20/'}


+char root_addr[] = "192.168.1.2";

+int poll_freq = 5000;
+int found_message = 0;
+int era = 2;

+ // code to regularily poll server, if we are doing that
+  int time = millis();
+  if (time / poll_freq > era) {
+    Serial.println(time);
+    era = time / poll_freq;
+    char tail_addr[] = "/hubs/poll/";
+    makeRequest(tail_addr);
+    if (found_message == 1) {
+      found_message = 0;
+      Serial.println("found message");
+      JustSwitchOn();
+      delay(3000); // method to open with polling
+      SwitchOff();
+    }
+  }

+void JustSwitchOn() {
+  digitalWrite(1+row, LOW);
+  digitalWrite(5+col, LOW);
+}
+
 void SwitchOff() {
   digitalWrite(1+row, HIGH);
   digitalWrite(5+col, HIGH);
 }
 
 +  int start_stream = 0;
   while (true) {
     if (client.available()) { // read incoming byes from server
       char c = client.read();
+      if (start_stream == 1) {
+        col = c - '0'; // this convert char into into
+        start_stream = 2;
+      } else if (start_stream == 2) {
+        row = c - '0';
+        found_message = 1;
+        start_stream = 3;
+      } else if (c == '?') {
+        start_stream = 1;
+      }
       Serial.print(c);
     }
 void makeRequest(char* tail) {
 
   Serial.println("connecting...");
 
   if (client.connect(root_addr, server_port)) { // could use server_ip
     Serial.println("connected");
     client.print("GET ");  // ex. "GET /hubs/connected/2 HTTP/1.1"
     client.print(tail);
     client.print(secretkey);
     client.println(" HTTP/1.1");
     client.println("Connection: close");
     client.println();
   }
   else {
     Serial.println("connection failed");
   }


### Initialize Hub with 'secret' id of 2 -- this will generate 2 x 4 locker objects associated with hub 2
### Simply visit this url to 'connect' the hub and generate locker objects if not yet present

In [34]:

# ip = 'http://127.0.0.1:8000'
connect = ip + '/hubs/connected/' + secret
poll = ip + '/hubs/poll/' + secret
finished = ip + '/hubs/finished/' + secret

In [35]:
resp = requests.get(connect); resp.text

'<html>Hi, arduino</html>\n'

In [36]:
resp = requests.get(poll); resp.text
action, column, row = tuple(char for char in resp.text)[:3]
action, column, row

('?', '1', '1')

In [37]:
# resp = requests.get(finished); resp.text

In [38]:
# action = '?'

In [39]:
def poll_status():
    resp = requests.get(poll); resp.text
    action, column, row = tuple(char for char in resp.text)[:3]
    return action, column, row


In [40]:
def open_locker(column, row):
    print('Opening {}-{}'.format(column, row))
    print('Waiting 2s for motor')
    time.sleep(2)

## Now POST an open object (simulate what the API/javascript will do)
### First get item data

In [41]:
# Must be logged in though session cookie for this to work
# Also must have a profile 1 and 2 in your local db
# Must have locker 1 free
# Profile 1 must have item 1 with no locker assigned
import json
item_data_json = requests.get(ip + '/api/owneditems/1/').text
item_data = json.loads(item_data_json)
item_data

{'actions': ['can_stock', 'can_delete'],
 'created_at': '2015-07-10T17:02:05.008779Z',
 'description': 'Ball for bowling',
 'id': 1,
 'locker': None,
 'owner': 1,
 'price': '10.00',
 'title': 'Bowling Ball'}

### Now assign that item to locker 1

In [42]:
item_data['locker'] = 1
resp = requests.put(ip + '/api/owneditems/1/', data=item_data)
print(resp.text)

{"id":1,"actions":["can_open"],"created_at":"2015-07-10T17:02:05.008779Z","title":"Bowling Ball","description":"Ball for bowling","price":"10.00","owner":1,"locker":1}


###Now create an unlock transaction
####This will also remove item 1 from the locker in the db
Note: If something goes awry, the unlock object keeps a record of which locker it had been in...

In [43]:
post_data = {
    "waiting": True,
    "profile": 1,
    "locker": 1
}
resp = requests.post(ip + '/api/unlocks/', data=post_data)
resp.text

'{"waiting":true,"profile":1,"locker":1}'

###Note the output in the console log for validation/setting up polling/destocking, POST

##Now we have an unlock action ready to be polled, so let's poll the server a couple of times with our 'dummy' client (simulate the raspberry pi python code)


In [44]:
for _ in range(2):
    action, column, row = poll_status()
    if action == '?':
        open_locker(column, row)
    else:
        print('Nothing to do yet...')
    time.sleep(1)

Opening 1-1
Waiting 2s for motor
Nothing to do yet...


#Success! It first opened 1-1 where the item was stored, and on the second poll, everything had cleared so there was nothing to do